# Think Green Investing 

## Imports

In [1]:
# Initial imports
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from datetime import datetime
import seaborn as sn
import matplotlib.pyplot as plt
import panel as pn

pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
from pathlib import Path
import plotly.graph_objects as go
import plotly.offline as pyo

from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
from numpy.random import seed
from tensorflow import random

%matplotlib inline

In [2]:
# Initializing alpaca trade api
load_dotenv()
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')
alpaca=tradeapi.REST(alpaca_api_key,alpaca_secret_key, api_version='v2')

# Data Collection

In [3]:
# Set the file paths for CSVs using the Path class from the pathlib library  
aqn_path = Path("aqn.csv")
csiq_path = Path ("csiq.csv")
cvx_path = Path("cvx.csv")
dq_path = Path("dq.csv")
fslr_path = Path ("fslr.csv")
sedg_path = Path("sedg.csv")
xom_path = Path("xom.csv")
sp500_path = Path("sp500.csv")
nasdaq_path = Path("nasdaq.csv")

In [4]:
# Read the data, set the `date` as a datetime index
aqn_df = pd.read_csv(aqn_path, index_col = "date", infer_datetime_format=True,  parse_dates=True)
csiq_df = pd.read_csv(csiq_path, index_col = "date", infer_datetime_format=True, parse_dates=True)
cvx_df = pd.read_csv(cvx_path, index_col = "date",  infer_datetime_format=True, parse_dates=True)
dq_df = pd.read_csv(dq_path, index_col = "date",  infer_datetime_format=True, parse_dates=True)
fslr_df = pd.read_csv(fslr_path,index_col = "date", infer_datetime_format=True, parse_dates=True)
sedg_df = pd.read_csv(sedg_path,index_col = "date", infer_datetime_format=True, parse_dates=True)
xom_df = pd.read_csv(xom_path, index_col = "date",  infer_datetime_format=True, parse_dates=True)
sp500_df= pd.read_csv(sp500_path, index_col = "date",  infer_datetime_format=True, parse_dates=True)
nasdaq_df = pd.read_csv(nasdaq_path, index_col = "date",  infer_datetime_format=True, parse_dates=True)

In [5]:
# Combine the dfs  
all_close_prices = pd.concat([aqn_df,csiq_df,cvx_df,dq_df,fslr_df,sedg_df,xom_df,sp500_df,nasdaq_df], axis= "rows", join= "inner")

# Reset index to date and create pivot table for stock closing prices by symbol 
all_close_prices= all_close_prices.reset_index()
all_close_prices = all_close_prices.pivot_table(values="close", index="date", columns="symbol")


# Display a few rows
all_close_prices.head()

symbol,AQN,CSIQ,CVX,DQ,FSLR,NASDAQ,SEDG,SP500,XOM
date,,,,,,,,,
2018-01-02 16:00:00,11.12,17.14,127.58,12.51,70.43,7006.90,38.05,2695.81,85.03
2018-01-03 16:00:00,10.78,17.07,128.51,12.63,69.80,7065.53,39.05,2713.06,86.70
2018-01-04 16:00:00,10.66,17.06,128.11,12.50,69.14,7077.92,37.35,2723.99,86.82
2018-01-05 16:00:00,10.87,17.01,127.90,12.96,69.17,7136.56,37.90,2743.15,86.75
2018-01-08 16:00:00,10.89,17.10,128.53,11.17,69.62,7157.39,38.35,2747.71,87.14


In [6]:
# Calculate daily returns of all stocks and indices  
all_returns = all_close_prices.pct_change()
all_returns = all_returns.dropna() 
all_returns.head()

symbol,AQN,CSIQ,CVX,DQ,FSLR,NASDAQ,SEDG,SP500,XOM
date,,,,,,,,,
2018-01-03 16:00:00,-0.030576,-0.004084,0.007290,0.009592,-0.008945,0.008367,0.026281,0.006399,0.019640
2018-01-04 16:00:00,-0.011132,-0.000586,-0.003113,-0.010293,-0.009456,0.001754,-0.043534,0.004029,0.001384
2018-01-05 16:00:00,0.019700,-0.002931,-0.001639,0.036800,0.000434,0.008285,0.014726,0.007034,-0.000806
2018-01-08 16:00:00,0.001840,0.005291,0.004926,-0.138117,0.006506,0.002919,0.011873,0.001662,0.004496
2018-01-09 16:00:00,-0.011938,-0.008772,-0.005368,0.073411,0.005745,0.000865,-0.029987,0.001303,-0.004246


In [7]:
# Calculate cumulative returns of all stocks
cumulative_returns = (1+ all_returns).cumprod()
cumulative_returns

symbol,AQN,CSIQ,CVX,DQ,FSLR,NASDAQ,SEDG,SP500,XOM
date,,,,,,,,,
2018-01-03 16:00:00,0.969424,0.995916,1.007290,1.009592,0.991055,1.008367,1.026281,1.006399,1.019640
2018-01-04 16:00:00,0.958633,0.995333,1.004154,0.999201,0.981684,1.010136,0.981603,1.010453,1.021051
2018-01-05 16:00:00,0.977518,0.992415,1.002508,1.035971,0.982110,1.018505,0.996058,1.017561,1.020228
2018-01-08 16:00:00,0.979317,0.997666,1.007446,0.892886,0.988499,1.021477,1.007884,1.019252,1.024815
2018-01-09 16:00:00,0.967626,0.988915,1.002038,0.958433,0.994179,1.022361,0.977661,1.020580,1.020463
...,...,...,...,...,...,...,...,...,...
2022-07-25 16:00:00,1.219424,1.957410,1.163819,4.825739,1.052818,1.681581,7.868068,1.471484,1.058215
2022-07-26 16:00:00,1.220324,1.881564,1.154726,4.823341,1.044583,1.650171,7.791853,1.454498,1.054099
2022-07-27 16:00:00,1.223921,1.987165,1.169933,5.150280,1.089876,1.717224,8.546386,1.492542,1.076914


In [8]:
# Cumulative returns for equal-weight Think Green Porfolio 
# Set initial investment
tg_initial_investment = 10000

# Set weights 
aqn_weight = 0.25
csiq_weight = 0.25
dq_weight = 0.25
fslr_weight = 0.25

# Calculate Think Green Porfolio returns
tg_portfolio_returns = aqn_weight*all_returns['AQN'] +  csiq_weight*all_returns['CSIQ'] +  dq_weight*all_returns['DQ'] +  fslr_weight*all_returns['FSLR']


# Use the `cumprod` function to calculate cumulative returns
tg_cumulative_returns = (1 + tg_portfolio_returns).cumprod()
tg_cumulative_returns

date
2018-01-03 16:00:00    0.991497
2018-01-04 16:00:00    0.983697
2018-01-05 16:00:00    0.996978
2018-01-08 16:00:00    0.965952
2018-01-09 16:00:00    0.980066
                         ...   
2022-07-25 16:00:00    2.617435
2022-07-26 16:00:00    2.587118
2022-07-27 16:00:00    2.697210
2022-07-28 16:00:00    2.852222
2022-07-29 16:00:00    2.964400
Length: 1151, dtype: float64

In [9]:
# Compute Think Green cumulative profit
tg_cumulative_profit = tg_initial_investment * tg_cumulative_returns
tg_cumulative_profit

date
2018-01-03 16:00:00     9914.969302
2018-01-04 16:00:00     9836.972975
2018-01-05 16:00:00     9969.779198
2018-01-08 16:00:00     9659.518070
2018-01-09 16:00:00     9800.660317
                           ...     
2022-07-25 16:00:00    26174.345658
2022-07-26 16:00:00    25871.183969
2022-07-27 16:00:00    26972.095598
2022-07-28 16:00:00    28522.223675
2022-07-29 16:00:00    29644.001454
Length: 1151, dtype: float64

In [10]:
# Cumulative returns for equal-weight Fossil Fuel Porfolio 
# Set initial investment
ff_initial_investment = 10000

cvx_weight = 0.5
xom_weight = 0.5

# Calculate Fossil Fuels portfolio returns
ff_returns = cvx_weight*all_returns['CVX'] +  xom_weight*all_returns['XOM']


# Use the `cumprod` function to calculate cumulative returns
ff_cumulative_returns = (1 + ff_returns).cumprod()
ff_cumulative_returns

date
2018-01-03 16:00:00    1.013465
2018-01-04 16:00:00    1.012589
2018-01-05 16:00:00    1.011351
2018-01-08 16:00:00    1.016115
2018-01-09 16:00:00    1.011230
                         ...   
2022-07-25 16:00:00    1.136218
2022-07-26 16:00:00    1.129570
2022-07-27 16:00:00    1.149232
2022-07-28 16:00:00    1.160297
2022-07-29 16:00:00    1.238816
Length: 1151, dtype: float64

In [11]:
# Compute Fossil Fuels cumulative profit
ff_cumulative_profit = ff_initial_investment * ff_cumulative_returns
ff_cumulative_profit

date
2018-01-03 16:00:00    10134.648354
2018-01-04 16:00:00    10125.889407
2018-01-05 16:00:00    10113.508066
2018-01-08 16:00:00    10161.149771
2018-01-09 16:00:00    10112.302902
                           ...     
2022-07-25 16:00:00    11362.184780
2022-07-26 16:00:00    11295.703198
2022-07-27 16:00:00    11492.322710
2022-07-28 16:00:00    11602.969251
2022-07-29 16:00:00    12388.162285
Length: 1151, dtype: float64

In [12]:
# Cumulative returns for NASDAQ index fund  
# Set initial investment
nq_initial_investment = 10000


# Calculate returns
nq_returns = all_returns['NASDAQ']


# Use the `cumprod` function to calculate cumulative returns
nq_cumulative_returns = (1 + nq_returns).cumprod()
nq_cumulative_returns

date
2018-01-03 16:00:00    1.008367
2018-01-04 16:00:00    1.010136
2018-01-05 16:00:00    1.018505
2018-01-08 16:00:00    1.021477
2018-01-09 16:00:00    1.022361
                         ...   
2022-07-25 16:00:00    1.681581
2022-07-26 16:00:00    1.650171
2022-07-27 16:00:00    1.717224
2022-07-28 16:00:00    1.735802
2022-07-29 16:00:00    1.768355
Name: NASDAQ, Length: 1151, dtype: float64

In [13]:
# Compute NASDAQ cumulative profit
nq_cumulative_profit = nq_initial_investment * nq_cumulative_returns
nq_cumulative_profit

date
2018-01-03 16:00:00    10083.674664
2018-01-04 16:00:00    10101.357234
2018-01-05 16:00:00    10185.046169
2018-01-08 16:00:00    10214.774008
2018-01-09 16:00:00    10223.608158
                           ...     
2022-07-25 16:00:00    16815.810130
2022-07-26 16:00:00    16501.705462
2022-07-27 16:00:00    17172.244502
2022-07-28 16:00:00    17358.018525
2022-07-29 16:00:00    17683.554782
Name: NASDAQ, Length: 1151, dtype: float64

In [14]:
# Cumulative returns for SP500 index fund  
# Set initial investment
sp500_initial_investment = 10000

# Calculate returns
sp500_returns = all_returns['SP500']

# Use the `cumprod` function to calculate cumulative returns
sp500_cumulative_returns = (1 + sp500_returns).cumprod()
sp500_cumulative_returns

date
2018-01-03 16:00:00    1.006399
2018-01-04 16:00:00    1.010453
2018-01-05 16:00:00    1.017561
2018-01-08 16:00:00    1.019252
2018-01-09 16:00:00    1.020580
                         ...   
2022-07-25 16:00:00    1.471484
2022-07-26 16:00:00    1.454498
2022-07-27 16:00:00    1.492542
2022-07-28 16:00:00    1.510652
2022-07-29 16:00:00    1.532115
Name: SP500, Length: 1151, dtype: float64

In [15]:
# Compute SP500 cumulative profit
sp500_cumulative_profit = sp500_initial_investment * sp500_cumulative_returns
sp500_cumulative_profit

date
2018-01-03 16:00:00    10063.988189
2018-01-04 16:00:00    10104.532589
2018-01-05 16:00:00    10175.605848
2018-01-08 16:00:00    10192.520986
2018-01-09 16:00:00    10205.800854
                           ...     
2022-07-25 16:00:00    14714.835244
2022-07-26 16:00:00    14544.979060
2022-07-27 16:00:00    14925.421302
2022-07-28 16:00:00    15106.517151
2022-07-29 16:00:00    15321.146520
Name: SP500, Length: 1151, dtype: float64